In [ ]:
# Pkg.add NPZ, PyPoy, Flux, Quaternions, MeshCat, CoordinateTransformations, Rotations, GeometryTypes, StaticArrays, DSP, MacroTools, StatsBase, Distributions, MultivariateStats, BSON

In [2]:
using LinearAlgebra, Statistics, Random
using NPZ
using PyPlot
using ArgCheck
using Flux
using ProgressMeter, Formatting 

In [3]:
using MeshCat
using Quaternions
using GeometryBasics
using CoordinateTransformations

In [20]:
include("../mocap-mtds/mocap_viz.jl")

Main.mocapviz

In [5]:
data_path = "data/"

#= Load from Numpy Compressed files dumped from python pre-processing =#
Ys = npzread(joinpath(data_path, "edin_Ys_30fps_final.npz"))
Us = npzread(joinpath(data_path, "edin_Us_30fps_final.npz"))

#= Extract from Dict --> Array of Matrices =#
N = length(Ys)
Ys = [Ys[string(i)] for i in 1:N]
Us = [Us[string(i)] for i in 1:N];

In [6]:

extract_joint_pos(y) = begin; @argcheck size(y,2) == 133; y[:,8:70]; end
extract_joint_rot(y) = begin; @argcheck size(y,2) == 133; y[:,71:133]; end
extract_trajectory(x) = begin; @argcheck size(x,2) == 150; x[:,1:48]; end

extract_trajectory (generic function with 1 method)

In [7]:
kwidentity(x; kwargs...) = identity(x)

kwidentity (generic function with 1 method)

In [8]:
std(reduce(vcat, Ys), dims=1)

1×67 Matrix{Float32}:
 1.00025  1.00014  1.00006  1.00025  1.00004  …  0.999999  1.00003  0.999905

In [9]:
qimag = Quaternions.imag
quat_list(x) = [quat(x[i,:]) for i in 1:size(x,1)]
quat_list_to_mat(x) = reduce(vcat, [qimag(xx)' for xx in x])
quaterion_angle_axis_w_y(θ) = quat(cos(θ/2), 0, sin(θ/2), 0)
apply_rotation(x, qrot) = qrot * x * conj(qrot)

apply_rotation (generic function with 1 method)

In [10]:
function reconstruct_positions(Y::Matrix; denormalize=false)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    if denormalize
        Y = Y .* loadYstd' .+ loadYmu'
    end
    
    root_r, root_x, root_z, joints = Y[:,1], Y[:,2], Y[:,3], Y[:,4:(63+3)]
    return _joints_fk(joints, root_x, root_z, root_r)
end

function _joints_fk(joints::Matrix{T}, root_x::Vector{T}, root_z::Vector{T}, 
        root_r::Vector{T}) where T <: Number

    n = size(joints, 1)
    joints = reshape(joints, n, 3, 21)
    joints = permutedims(joints, [1,3,2])
    rotation = Quaternion(1.0)
    offsets = []
    translation = zeros(3)

    for i = 1:n
        joints[i,:,:] = apply_rotation(quat_list(joints[i,:,:]), rotation) |> quat_list_to_mat
        joints[i,:,1] = joints[i,:,1] .+ translation[1]
        joints[i,:,3] = joints[i,:,3] .+ translation[3]
        
        rotation = quaterion_angle_axis_w_y(-root_r[i]) * rotation
        append!(offsets, apply_rotation(quat(0.,0,0,1), rotation))
        translation = translation + qimag(apply_rotation(quat(0., root_x[i], 0, root_z[i]), rotation))
    end
    
    return joints
end

_joints_fk (generic function with 1 method)

In [11]:
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/alexbird/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [17]:
@which Transformation

CoordinateTransformations

In [12]:
cur_recon = reconstruct_positions(Ys[1][1:300,:]);

In [21]:
vis = mocapviz.create_animation([cur_recon], "test"; vis=vis, linemesh=[mocapviz.yellowmesh])

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [17]:
using GeometryBasics: Cylinder, HyperSphere, Point

In [ ]:
AbstractGeometry, AbstractMesh, MeshFileGeometry

In [54]:
@which GeometryLike

MeshCat

In [44]:
Base.show_supertypes(typeof(HyperRectangle(Vec(0., 0, 0), Vec(1., 1, 1))))

HyperRectangle{3, Float64} <: GeometryPrimitive{3, Float64} <: AbstractGeometry{3, Float64} <: Any

In [53]:
typeof(HyperRectangle(Vec(0., 0, 0), Vec(1., 1, 1))) <: AbstractGeometry

true

In [18]:
setobject!(vis, HyperRectangle(Vec(0., 0, 0), Vec(1., 1, 1)))
settransform!(vis, Translation(-0.5, -0.5, 0))

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [41]:
create_animation([cur_recon], "test"; vis=vis, linemesh=[mocapviz.yellowmesh])

LoadError: UndefVarError: anim_settransform! not defined

In [34]:
# using mocapviz: gen_numstr, LineCollection, JointCollection, setobj_collection!, update_pos!, settransform_collection!
using DSP

const LineCollection = mocapviz.LineCollection;
const JointCollection = mocapviz.JointCollection;
const setobj_collection! = mocapviz.setobj_collection!;
const update_pos! = mocapviz.update_pos!;
const settransform_collection! = mocapviz.settransform_collection!

settransform_collection! (generic function with 3 methods)

In [38]:
function create_animation(data::Vector, names::Union{String, Array{String}}="dataset";
    vis=nothing, parents=[1,2,3,4,1,6,7,8,1,10,11,12,12,14,15,16,12,18,19,20],
    jointmesh::Union{AbstractMaterial, Vector{T} where T <: AbstractMaterial}=mocapviz.greymesh,
    linemesh::Union{AbstractMaterial, Vector{T} where T <: AbstractMaterial}=mocapviz.yellowmesh, scale=0.1,
    camera::Symbol=:front, path::Union{Nothing, AbstractMatrix{T} where T <: Real, Vector}=nothing)

    # dimensions
    Ts = [size(d,1) for d in data]
    ls = [size(d,2) for d in data]
    n  = length(ls)
    maxT = maximum(Ts)
    ispath = !(path === nothing)

    # convenience expansions if args are singular
    (names isa String) && (names = [names * string(i) for i in 1:n])
    (jointmesh isa AbstractMaterial) && (jointmesh = repeat([jointmesh], n))
    (linemesh isa AbstractMaterial) && (linemesh = repeat([linemesh], n))
    ispath && (path isa AbstractMatrix) && (path = [path])
    npaths = ispath ? length(path) : 0
    (npaths >= 1) && (pathmesh = map(darkencol, linemesh));
    (npaths == 1) && (pathmesh = [blackmesh]);
    # assertions
    @argcheck length(data) == length(names)
    @assert (all([ndims(d) for d in data] .== 3) && all([size(d,3) for d in data] .==3)) "Need NxJx3 matrices."
    @argcheck camera in [:front, :back, :static]
    ispath && @argcheck size(path[1]) == (Ts[1], 12)
    # @assert all(ls .<= length(parents)-1) "more joints given (dim 1) than are assigned parents."
    # any(ls .> length(parents)) && @warn format("({:d}/{:d}) datasets have fewer joints than specified in parents.",
    #     sum(ls .> length(parents)), n)

    if vis === nothing
        vis = Visualizer()
        open(vis)
    else
        @assert (vis isa Visualizer) "Don't know what to do with vis::$(typeof(vis)). Expecting Visualizer."
    end

    gen_numstr(prefix) = num -> prefix * format("{:02d}", num);
    dotstr, linestr, pathstr = gen_numstr("dot_"), gen_numstr("line_"), gen_numstr("path_")

    # Initialise objects for each dataset
    objs = map(1:n) do i
        if i <= npaths
            obj_path = LineCollection(repeat([Cylinder(zero(Point{3, Float64}), Point{3}([1.,0,0]), 0.02)], 10), map(pathstr,  1:10), repeat([pathmesh[i]], 10))
            setobj_collection!(vis[names[i]], obj_path)
        else
            obj_path = nothing
        end

        nj, nl = ls[i], ls[i]-1
        obj_lc = LineCollection(repeat([Cylinder(zero(Point{3, Float64}), Point{3}([1.,0,0]), 0.03)], nl), map(linestr, 1:nl), repeat([linemesh[i]], nl))
        obj_jc = JointCollection(repeat([HyperSphere(Point{3}(0.), 0.04)], nj), map(dotstr, 1:nj), repeat([jointmesh[i]], nj))
        setobj_collection!(vis[names[i]], obj_lc, obj_jc)  # note order: lines at back, joints at front
        (obj_lc, obj_jc, obj_path)
    end

    #=========== BEGIN ANIMATION CONSTRUCTION ===============================#
    anim = Animation()

    # camera trajectory (smoothed (MA-4) over skeleton traj)
    cam_ix = 1
    pos = hcat([data[cam_ix][tt,1,1]*scale for tt in 1:maxT], [data[cam_ix][tt,1,3]*scale for tt in 1:maxT])
    pos = hcat(DSP.conv(pos[:,1], Windows.rect(4)/4), DSP.conv(pos[:,2], Windows.rect(4)/4))[1:maxT,:]
    cc = [scale, scale] * (camera == :front ? -1 : +1)

    reord = [1,3,2] # need to permute z/y axis for three.js setup
    for tt in 1:maxT
        atframe(anim, tt-1) do
            for i in 1:n
                if Ts[i] >= tt
                    lines, joints, pathlines = objs[i]
                    update_pos!(lines, data[i][tt,:,reord]*scale, parents)   # Note that update_pos only updates the line/joint collection position.
                    update_pos!(joints, data[i][tt,:,reord]*scale);          # Once these objects are correct, `settransform_collection` pushes => animation.
                    settransform_collection!(vis[names[i]], lines, joints; anim=true)  # note order: lines at back, joints at front

                    if i <= npaths
                        trail = randn(5,3)*0.001   # case i=1: all segments need positive length (Meshcat reqmt.)
                        trail_ixs = max(tt-41, 1):10:(tt-1)
                        if false
                            trail[(6-length(trail_ixs)):5, :] = project_ground(data[i][trail_ixs, 1, reord])
                        else
                            trail[(6-length(trail_ixs)):5, :] = hcat(path[i][trail_ixs, 1], path[i][trail_ixs, 7], zeros(length(trail_ixs)))
                        end
                        path_matrix = vcat(trail, hcat(reshape(path[i][tt, :], 6, 2), zeros(6)))*scale
                        update_pos!(pathlines, path_matrix[1:10,:], path_matrix[2:11,:])
                        settransform_collection!(vis[names[i]], pathlines; anim=true)
                    end
                end
            end
            # one-per-frame updates
            if camera != :static
                settransform!(vis["/Cameras/default"], Translation(pos[tt,1] + cc[1], pos[tt,2] + cc[2], 1))
            end
        end
    end

    setanimation!(vis, anim)
    return vis
end

create_animation (generic function with 2 methods)